In [20]:
from itertools import compress
import matplotlib.pyplot as plt
import numpy as np
import os
import mne
import mne_nirs
import snirf
from mne_nirs.statistics import run_glm
import statsmodels.formula.api as smf
import pandas as pd

# Function to extract trial name from filename
def extract_trial_name(snirf_file):
    filename = os.path.basename(snirf_file)
    filename_parts = filename.replace('.snirf', '').split('_')
    
    if 'Keehn' in filename_parts and 'Logan' in filename_parts:
        trial = "MRSA"
    elif 'Will' in filename_parts and 'Rudd' in filename_parts:
        trial = 'none'
    elif 'Timothy' in filename_parts and 'Yang' in filename_parts:
        trial = 'R'
    else:
        trial = 'unknown'
    
    return trial

def process_single_file(snirf_file, participant_id):
    """Process a single SNIRF file through the complete pipeline"""
    
    trial_name = extract_trial_name(snirf_file)
    print(f"Loading data from: {snirf_file} (Trial: {trial_name})")
    
    # Load and process data (following your original cells 1-12)
    raw_intensity = mne.io.read_raw_snirf(snirf_file, optode_frame='unknown', verbose="info")
    raw_intensity.load_data()

    return raw_intensity, trial_name
    
snirf_file1 = "..\\data\\raw\\250624_1014_20250624_Keehn_Logan_NS_Keehn_Logan_concat.snirf"
raw_intensity, trial_name = process_single_file(snirf_file1, 1)
# Remove thinkaloud annotations
raw_intensity.annotations.delete([index for index,value in enumerate(raw_intensity.annotations.description) if 'thinkaloud' in value])

# Convert to hemoglobin
raw_od = mne.preprocessing.nirs.optical_density(raw_intensity, verbose=False)
raw_haemo = mne.preprocessing.nirs.beer_lambert_law(raw_od, ppf=0.1)



Loading data from: ..\data\raw\250624_1014_20250624_Keehn_Logan_NS_Keehn_Logan_concat.snirf (Trial: MRSA)
Loading c:\Users\jdtull\Documents\CRED\Projects\Design Cognition\Design-Cognition\cred_fnirs_toolkit\notebooks\..\data\raw\250624_1014_20250624_Keehn_Logan_NS_Keehn_Logan_concat.snirf

Loading c:\Users\jdtull\Documents\CRED\Projects\Design Cognition\Design-Cognition\cred_fnirs_toolkit\notebooks\..\data\raw\250624_1014_20250624_Keehn_Logan_NS_Keehn_Logan_concat.snirf
Found jitter of 0.000000% in sample times.
Found jitter of 0.000000% in sample times.
Reading 0 ... 19631  =      0.000 ...  2453.875 secs...
Reading 0 ... 19631  =      0.000 ...  2453.875 secs...


C:\Users\jdtull\AppData\Local\Temp\ipykernel_31992\1413695755.py:46: RuntimeWarning: Negative intensities encountered. Setting to abs(x)
  raw_od = mne.preprocessing.nirs.optical_density(raw_intensity, verbose=False)


In [21]:

# Extract averages for each block, trial, and channel
def get_hemoglobin_averages(raw_haemo, trial_name):
    """
    Extract average oxy and deoxy hemoglobin for each block and channel
    
    Parameters:
    -----------
    raw_haemo : mne.io.Raw
        Hemoglobin data (converted with beer_lambert_law)
    trial_name : str
        Name of the trial
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with average oxy/deoxy hemoglobin per block/channel
    """
    
    # Get the data
    data = raw_haemo.get_data()  # shape: (n_channels, n_times)
    ch_names = raw_haemo.ch_names
    
    # Get event information if available
    events, event_ids = mne.events_from_annotations(raw_haemo)
    
    # Create results list
    results = []
    
    if len(events) > 0:
        # Process each event/block
        for event_idx, (event_sample, duration, event_id) in enumerate(events):
            # Get event label
            event_label = None
            for label, id_val in event_ids.items():
                if id_val == event_id:
                    event_label = label
                    break
            
            # Define block window (e.g., from event to next event or fixed duration)
            block_start = event_sample
            if event_idx < len(events) - 1:
                block_end = events[event_idx + 1][0]
            else:
                block_end = len(data[0])
            
            # Extract data for this block
            block_data = data[:, block_start:block_end]
            
            # Calculate mean for each channel
            for ch_idx, ch_name in enumerate(ch_names):
                # Determine if it's HbO or HbR
                if 'hbo' in ch_name.lower():
                    hb_type = 'HbO'
                elif 'hbr' in ch_name.lower():
                    hb_type = 'HbR'
                else:
                    hb_type = 'Unknown'
                
                mean_value = np.mean(block_data[ch_idx])
                
                results.append({
                    'Trial': trial_name,
                    'Block': event_label if event_label else f'Block_{event_idx}',
                    'Channel': ch_name,
                    'HbType': hb_type,
                    'Mean_Value': mean_value,
                    'Std_Value': np.std(block_data[ch_idx]),
                    'Block_Duration_s': (block_end - block_start) / raw_haemo.info['sfreq']
                })
    else:
        # If no events, treat entire recording as one block
        for ch_idx, ch_name in enumerate(ch_names):
            if 'hbo' in ch_name.lower():
                hb_type = 'HbO'
            elif 'hbr' in ch_name.lower():
                hb_type = 'HbR'
            else:
                hb_type = 'Unknown'
            
            mean_value = np.mean(data[ch_idx])
            
            results.append({
                'Trial': trial_name,
                'Block': 'Entire_Recording',
                'Channel': ch_name,
                'HbType': hb_type,
                'Mean_Value': mean_value,
                'Std_Value': np.std(data[ch_idx]),
                'Block_Duration_s': len(data[0]) / raw_haemo.info['sfreq']
            })
    
    return pd.DataFrame(results)

# Process the first file
averages_df = get_hemoglobin_averages(raw_haemo, trial_name)
print(f"\nAverages for trial '{trial_name}':")
print(averages_df.head(10))


Used Annotations descriptions: [np.str_('Design problem context'), np.str_('Generate stakeholder needs'), np.str_('Prioritize stakeholder needs'), np.str_('Stakeholder identification'), np.str_('Stakeholder personas')]

Averages for trial 'MRSA':
  Trial                   Block     Channel HbType    Mean_Value  Std_Value  \
0  MRSA  Design problem context   S8_D1 hbo    HbO -7.613588e-05   0.000079   
1  MRSA  Design problem context   S1_D4 hbo    HbO -5.629119e-05   0.000053   
2  MRSA  Design problem context   S8_D9 hbo    HbO -9.734764e-05   0.000047   
3  MRSA  Design problem context  S2_D25 hbo    HbO -8.318616e-05   0.000086   
4  MRSA  Design problem context   S3_D1 hbo    HbO -4.131784e-05   0.000067   
5  MRSA  Design problem context   S2_D8 hbo    HbO -4.119831e-06   0.000079   
6  MRSA  Design problem context   S7_D5 hbo    HbO  3.398950e-04   0.000052   
7  MRSA  Design problem context   S6_D4 hbo    HbO -1.011077e-04   0.000046   
8  MRSA  Design problem context  S3_D26 hb

In [22]:

# Process all SNIRF files
snirf_files = [
    "..\\data\\raw\\250624_1014_20250624_Keehn_Logan_NS_Keehn_Logan_concat.snirf",
    "..\\data\\raw\\250625_1408_20250625_Rudd_Will_NS_rudd_will.snirf",
    "..\\data\\raw\\250630_1555_20250630_Yang_Timothy_NS_yang_tim.snirf"
]

all_averages = []

for participant_id, snirf_file in enumerate(snirf_files, start=1):
    try:
        raw_intensity, trial_name = process_single_file(snirf_file, participant_id)
        
        # Remove thinkaloud annotations
        thinkaloud_indices = [index for index, value in enumerate(raw_intensity.annotations.description) if 'thinkaloud' in value]
        if thinkaloud_indices:
            raw_intensity.annotations.delete(thinkaloud_indices)
        
        # Convert to hemoglobin
        raw_od = mne.preprocessing.nirs.optical_density(raw_intensity, verbose=False)
        raw_haemo = mne.preprocessing.nirs.beer_lambert_law(raw_od, ppf=0.1)
        
        # Get averages
        averages_df = get_hemoglobin_averages(raw_haemo, trial_name)
        averages_df['Participant_ID'] = participant_id
        all_averages.append(averages_df)
        
        print(f"\nProcessed {participant_id}: {trial_name} ({len(averages_df)} measurements)")
        
    except Exception as e:
        print(f"\nError processing file {participant_id}: {e}")

# Combine all results
combined_df = pd.concat(all_averages, ignore_index=True)

# Reorder columns for better readability
column_order = ['Participant_ID', 'Trial', 'Block', 'Channel', 'HbType', 'Mean_Value', 'Std_Value', 'Block_Duration_s']
combined_df = combined_df[column_order]

print(f"\n{'='*80}")
print(f"Combined Results: {len(combined_df)} total measurements")
print(f"{'='*80}")
print(combined_df.head(20))
print(f"\nUnique Trials: {combined_df['Trial'].unique()}")
print(f"Unique HbTypes: {combined_df['HbType'].unique()}")


Loading data from: ..\data\raw\250624_1014_20250624_Keehn_Logan_NS_Keehn_Logan_concat.snirf (Trial: MRSA)
Loading c:\Users\jdtull\Documents\CRED\Projects\Design Cognition\Design-Cognition\cred_fnirs_toolkit\notebooks\..\data\raw\250624_1014_20250624_Keehn_Logan_NS_Keehn_Logan_concat.snirf
Found jitter of 0.000000% in sample times.
Reading 0 ... 19631  =      0.000 ...  2453.875 secs...


C:\Users\jdtull\AppData\Local\Temp\ipykernel_31992\1821995662.py:20: RuntimeWarning: Negative intensities encountered. Setting to abs(x)
  raw_od = mne.preprocessing.nirs.optical_density(raw_intensity, verbose=False)


Used Annotations descriptions: [np.str_('Design problem context'), np.str_('Generate stakeholder needs'), np.str_('Prioritize stakeholder needs'), np.str_('Stakeholder identification'), np.str_('Stakeholder personas')]

Processed 1: MRSA (2040 measurements)
Loading data from: ..\data\raw\250625_1408_20250625_Rudd_Will_NS_rudd_will.snirf (Trial: none)
Loading c:\Users\jdtull\Documents\CRED\Projects\Design Cognition\Design-Cognition\cred_fnirs_toolkit\notebooks\..\data\raw\250625_1408_20250625_Rudd_Will_NS_rudd_will.snirf
Found jitter of 0.000000% in sample times.


C:\Users\jdtull\AppData\Local\Temp\ipykernel_31992\1413695755.py:35: RuntimeWarning: Extraction of measurement date from SNIRF file failed. The date is being set to January 1st, 2000, instead of 2025-06-25
  raw_intensity = mne.io.read_raw_snirf(snirf_file, optode_frame='unknown', verbose="info")


Reading 0 ... 17540  =      0.000 ...  2155.315 secs...


...

Used Annotations descriptions: [np.str_('Design problem context'), np.str_('Generate stakeholder needs'), np.str_('Prioritize stakeholder needs'), np.str_('Stakeholder identification'), np.str_('Thinkaloud practice AUT'), np.str_('Thinkaloud practice basic math'), np.str_('Thinkalound practice anagram')]

Processed 2: none (2856 measurements)
Loading data from: ..\data\raw\250630_1555_20250630_Yang_Timothy_NS_yang_tim.snirf (Trial: R)
Loading c:\Users\jdtull\Documents\CRED\Projects\Design Cognition\Design-Cognition\cred_fnirs_toolkit\notebooks\..\data\raw\250630_1555_20250630_Yang_Timothy_NS_yang_tim.snirf
Found jitter of 0.000000% in sample times.


C:\Users\jdtull\AppData\Local\Temp\ipykernel_31992\1413695755.py:35: RuntimeWarning: Extraction of measurement date from SNIRF file failed. The date is being set to January 1st, 2000, instead of 2025-06-30
  raw_intensity = mne.io.read_raw_snirf(snirf_file, optode_frame='unknown', verbose="info")


Reading 0 ... 16650  =      0.000 ...  2045.957 secs...


C:\Users\jdtull\AppData\Local\Temp\ipykernel_31992\1821995662.py:20: RuntimeWarning: Negative intensities encountered. Setting to abs(x)
  raw_od = mne.preprocessing.nirs.optical_density(raw_intensity, verbose=False)


Used Annotations descriptions: [np.str_('Design problem context'), np.str_('Generate stakeholder needs'), np.str_('Prioritize stakeholder needs'), np.str_('Stakeholder identification'), np.str_('Stakeholder personas')]

Processed 3: R (2040 measurements)

Combined Results: 6936 total measurements
    Participant_ID Trial                   Block      Channel HbType  \
0                1  MRSA  Design problem context    S8_D1 hbo    HbO   
1                1  MRSA  Design problem context    S1_D4 hbo    HbO   
2                1  MRSA  Design problem context    S8_D9 hbo    HbO   
3                1  MRSA  Design problem context   S2_D25 hbo    HbO   
4                1  MRSA  Design problem context    S3_D1 hbo    HbO   
5                1  MRSA  Design problem context    S2_D8 hbo    HbO   
6                1  MRSA  Design problem context    S7_D5 hbo    HbO   
7                1  MRSA  Design problem context    S6_D4 hbo    HbO   
8                1  MRSA  Design problem context   S3_

In [23]:

# Create pivot table: Trials (rows) x Blocks (columns) with HbO averages
# Filter for HbO only
hbo_df = combined_df[combined_df['HbType'] == 'HbO'].copy()

# Create a unique identifier combining Trial, Participant, and Channel for rows
hbo_df['Row_Label'] = hbo_df['Trial'] + '_P' + hbo_df['Participant_ID'].astype(str) + '_' + hbo_df['Channel'].astype(str)

# Pivot: Rows = Trial/Participant/Channel, Columns = Blocks, Values = Mean HbO
pivot_table = hbo_df.pivot_table(
    index='Row_Label',
    columns='Block',
    values='Mean_Value',
    aggfunc='mean'
)

print("\n" + "="*80)
print("HbO Averages Pivot Table (Trials × Blocks)")
print("="*80)
print(pivot_table)

# Save to CSV
output_csv = "..\\data\\processed\\hbo_averages_pivot.csv"
os.makedirs(os.path.dirname(output_csv), exist_ok=True)
pivot_table.to_csv(output_csv)
print(f"\nSaved to: {output_csv}")

# Alternative: Simpler version with Trial as rows (averaged across channels)
pivot_simple = hbo_df.pivot_table(
    index='Trial',
    columns='Block',
    values='Mean_Value',
    aggfunc='mean'
)

print("\n" + "="*80)
print("Simplified HbO Averages (Trial-averaged across all channels)")
print("="*80)
print(pivot_simple)

# Save simplified version
output_csv_simple = "..\\data\\processed\\hbo_averages_pivot_simple.csv"
pivot_simple.to_csv(output_csv_simple)
print(f"\nSaved to: {output_csv_simple}")



HbO Averages Pivot Table (Trials × Blocks)
Block                Design problem context  Generate stakeholder needs  \
Row_Label                                                                 
MRSA_P1_S10_D10 hbo               -0.000199                   -0.000055   
MRSA_P1_S10_D13 hbo               -0.000206                   -0.000035   
MRSA_P1_S10_D14 hbo               -0.000112                   -0.000070   
MRSA_P1_S10_D15 hbo               -0.000079                   -0.000082   
MRSA_P1_S10_D16 hbo               -0.000163                   -0.000047   
...                                     ...                         ...   
none_P2_S9_D14 hbo                -0.000133                   -0.000009   
none_P2_S9_D15 hbo                -0.000090                   -0.000024   
none_P2_S9_D16 hbo                -0.000143                   -0.000037   
none_P2_S9_D4 hbo                 -0.000059                   -0.000065   
none_P2_S9_D9 hbo                  0.000002             